In [0]:
import numpy as np
import pandas as pd
 
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import mlflow
import mlflow.sklearn
from hyperopt import fmin, tpe, hp, SparkTrials, STATUS_OK

Loading the data
Displaying  the first 5 rows of the dataset
link=-https://www.kaggle.com/santoshd3/bank-customers?select=Churn+Modeling.csv

In [0]:
cust_attrition_data = pd.read_csv('/dbfs/FileStore/tables/Bank_customer.csv')

cust_attrition_data.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,Yes,Yes,101348.88,Yes
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,No,Yes,112542.58,No
2,15619304,Onio,502,France,Female,42,8,159660.80,3,Yes,No,113931.57,Yes
3,15701354,Boni,699,France,Female,39,1,0.00,2,No,No,93826.63,No
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,Yes,Yes,79084.10,No


In [0]:
cust_attrition_data.shape

Out[3]: (10000, 13)

In [0]:
cust_attrition_data.columns

Out[5]: Index(['CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender', 'Age',
       'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Exited'],
      dtype='object')

Checking unique values for each column.Customerid and Surname can be dropped

In [0]:
cust_attrition_data.nunique()

Out[4]: CustomerId         10000
Surname             2932
CreditScore          460
Geography              3
Gender                 3
Age                   70
Tenure                11
Balance             6382
NumOfProducts          4
HasCrCard              2
IsActiveMember         2
EstimatedSalary     9999
Exited                 2
dtype: int64

Features data and target data are made into separate dataframes

In [0]:
X = cust_attrition_data.drop(['CustomerId', 'Surname', 'Exited'], axis = 1)
y = cust_attrition_data['Exited']

Target variable is converted into numeric form

In [0]:
class_dict = {'No': 0, 'Yes': 1}

y = y.replace(class_dict)
y

Out[6]: 0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

Data is imbalanced

In [0]:
y.value_counts()

Out[7]: 0    7963
1    2037
Name: Exited, dtype: int64

We are going to one hot encode categorical columns with dropping first category.

In [0]:
categoricalCols = ['Geography', 'Gender', 'HasCrCard', 'IsActiveMember']
numericCols = ['Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

categorical_transformer = OneHotEncoder(drop = 'first')

Preprocessing steps are defined with ColumnTransformer which will one hot encode only categorical colums and scale remainder numeric columns

In [0]:
preprocessor = ColumnTransformer(
    transformers = [('cat', categorical_transformer, categoricalCols)], 
    remainder = StandardScaler() 
)

print(preprocessor)

ColumnTransformer(remainder=StandardScaler(),
                  transformers=[('cat', OneHotEncoder(drop='first'),
                                 ['Geography', 'Gender', 'HasCrCard',
                                  'IsActiveMember'])])


In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify = y)

X_train.shape, X_test.shape

Out[10]: ((7000, 10), (3000, 10))

Creating and train a decision tree classifier model and by using MLFLOW we are logging 
 parameters, metrics, and the model.

In [0]:
with mlflow.start_run():
 
    criterion= 'entropy'
    max_depth = 5
    max_features = 5
    
    dtc = DecisionTreeClassifier(criterion = criterion, max_depth = max_depth,
                                 max_features = max_features)

    pipeline = Pipeline( steps = [('preprocessor', preprocessor), ('classifier', dtc)])
    pipeline.fit(X_train, y_train)

    predictions =  pipeline.predict(X_test) 
    accuracy = accuracy_score(y_test, predictions)
    precisionscore = precision_score(y_test, predictions)
    recallscore = recall_score(y_test, predictions)
    f1score = f1_score(y_test, predictions)
  
    mlflow.log_param('criterion', criterion)
    mlflow.log_param('max_depth', max_depth)
    mlflow.log_param('max_features', max_features)
  
    mlflow.log_metric('F1score',  f1score)
    mlflow.log_metric('Recall_score',  recallscore)
    mlflow.log_metric('Precision_score',  precisionscore)
    mlflow.log_metric('Accuracy_score',  accuracy)

    mlflow.sklearn.log_model(dtc, 'dtc_model')

In [0]:
from hyperopt.pyll import scope

def objective(params):
    
    with mlflow.start_run(nested = True):

        clf = DecisionTreeClassifier(max_features = params['max_features'], 
                                     max_depth = params['max_depth'],
                                     criterion = params['criterion'])
        pipeline = Pipeline(steps = [('preprocessor', preprocessor), ('model', clf)])
        pipeline.fit(X_train, y_train)
        
        predictions =  pipeline.predict(X_test) 
        accuracy = accuracy_score(y_test, predictions)
        precisionscore = precision_score(y_test, predictions)
        recallscore = recall_score(y_test, predictions)
        f1score = f1_score(y_test, predictions)

        mlflow.log_metric('F1score',  f1score)
        mlflow.log_metric('Recall_score',  recallscore)
        mlflow.log_metric('Precision_score',  precisionscore)
        mlflow.log_metric('Accuracy_score',  accuracy)

        mlflow.sklearn.log_model(dtc, 'dtc_hpo')
  
        return {'loss': -f1score, 'status': STATUS_OK}

Search space for hyperparameters is defined

In [0]:
search_space = {'max_features': scope.int(hp.quniform('max_features', 1, 10, 1)),
                'max_depth': scope.int(hp.quniform('max_depth', 1, 15, 1)),
                'criterion': hp.choice('criterion', ['gini', 'entropy'])} 

Algorithm is defined

The two main choices are:

hyperopt.tpe.suggest: Tree of Parzen Estimators, a Bayesian approach which iteratively and adaptively selects new hyperparameter settings to explore based on past results
hyperopt.rand.suggest: Random search, a non-adaptive approach that samples over the search space

In [0]:
algo = tpe.suggest

In [0]:
argmin = fmin(
  fn = objective,
  space = search_space,
  algo = algo,
  max_evals = 16)

print('Best value found: ', argmin)

100%|██████████| 16/16 [01:03<00:00,  3.96s/trial, best loss: -0.5540166204986151]
Best value found:  {'criterion': 0, 'max_depth': 9.0, 'max_features': 9.0}


In [0]:
from hyperopt import SparkTrials
 
spark_trials = SparkTrials()
 
with mlflow.start_run():
    argmin = fmin(
    fn = objective,
    space = search_space,
    algo = algo,
    max_evals = 16,
    trials = spark_trials)

print('Best value found: ', argmin)

Because the requested parallelism was None or a non-positive value, parallelism will be set to (4), which is Spark's default parallelism (4), or 1, whichever is greater. We recommend setting parallelism explicitly to a positive value because the total of Spark task slots is subject to cluster sizing.
Hyperopt with SparkTrials will automatically track trials in MLflow. To view the MLflow experiment associated with the notebook, click the 'Runs' icon in the notebook context bar on the upper right. There, you can view all runs.
To view logs from trials, please check the Spark executor logs. To view executor logs, expand 'Spark Jobs' above until you see the (i) icon next to the stage from the trial job. Click it and find the list of tasks. Click the 'stderr' link for a task to view trial logs.
100%|██████████| 16/16 [00:44<00:00,  2.77s/trial, best loss: -0.5822784810126582]
Total Trials: 16: 16 succeeded, 0 failed, 0 cancelled.
Best value found:  {'criterion': 0, 'max_depth': 7.0, 'max_fe

In [0]:
spark_trials.results

Out[17]: [{'loss': -0.5822784810126582, 'status': 'ok'},
 {'loss': -0.5004721435316336, 'status': 'ok'},
 {'loss': -0.0, 'status': 'ok'},
 {'loss': -0.5549845837615621, 'status': 'ok'},
 {'loss': -0.45837615621788286, 'status': 'ok'},
 {'loss': -0.33834586466165417, 'status': 'ok'},
 {'loss': -0.3480278422273782, 'status': 'ok'},
 {'loss': -0.48462929475587707, 'status': 'ok'},
 {'loss': -0.5256673511293634, 'status': 'ok'},
 {'loss': -0.5088757396449705, 'status': 'ok'},
 {'loss': -0.0, 'status': 'ok'},
 {'loss': -0.526810912511759, 'status': 'ok'},
 {'loss': -0.27114093959731544, 'status': 'ok'},
 {'loss': -0.51295799821269, 'status': 'ok'},
 {'loss': -0.5483568075117371, 'status': 'ok'},
 {'loss': -0.52565445026178, 'status': 'ok'}]

Now, we examine four algorithms available in scikit-learn: support vector machines (SVM), random forest, and logistic regression and Decison tree

In the following cell, we are defining  a parameter params['type'] for the model name. This function also runs the training and calculates the cross-validation accuracy.

In [0]:
def objective(params):

    with mlflow.start_run(nested = True):
        classifier_type = params['type']
        del params['type']
        
        if classifier_type == 'svm':
            clf = SVC(**params)
        elif classifier_type == 'rf':
            clf = RandomForestClassifier(**params)
        elif classifier_type == 'logreg':
            clf = LogisticRegression(**params)
        elif classifier_type == 'dtc':
            clf = DecisionTreeClassifier(**params)
        else:
            return 0
        
        pipeline = Pipeline(steps = [('preprocessor', preprocessor), ('model', clf)])
        pipeline.fit(X_train, y_train)
        
        predictions =  pipeline.predict(X_test) 
        accuracy = accuracy_score(y_test, predictions)
        precisionscore = precision_score(y_test, predictions)
        recallscore = recall_score(y_test, predictions)
        f1score = f1_score(y_test, predictions)
        
        mlflow.log_metric('F1score',  f1score)
        mlflow.log_metric('Recall_score',  recallscore)
        mlflow.log_metric('Precision_score',  precisionscore)
        mlflow.log_metric('Accuracy_score',  accuracy)
        
        mlflow.sklearn.log_model(clf, 'clf_hpo')

        return {'loss': -f1score, 'status': STATUS_OK}

Search space is defined for multiple models
Using hp.choice to select different models.

In [0]:
search_space = hp.choice('classifier_type', [
    {
        'type': 'svm',
        'C': hp.lognormal('SVM_C', 0, 1.0),
        'kernel': hp.choice('kernel', ['linear', 'rbf'])
    },
    {
        'type': 'rf',
        'n_estimators':scope.int(hp.quniform('n_estimators', 100, 500, 50)),
        'max_depth': scope.int(hp.quniform('max_depth_rf', 2, 20 , 1)),
        'criterion': hp.choice('criterion_rf', ['gini', 'entropy'])
    },
    {
        'type': 'logreg',
        'C': hp.lognormal('LR_C', 0, 1.0),
        'solver': hp.choice('solver', ['liblinear', 'lbfgs'])
    },
  
    {
        'type': 'dtc',
        'max_features':scope.int(hp.quniform('max_features', 1,10,1)),
        'max_depth': scope.int(hp.quniform('max_depth_dtc', 2, 20, 1)),
        'criterion': hp.choice('criterion_dtc', ['gini', 'entropy'])
    }
    
    
])

Same steps are repeated as done for single model case. This time Best accuracy obtained is around 86% with RF model with hyperparameters {'criterion': 'gini', 'max_depth': 11, 'n_estimators': 100, 'type': 'rf'}

In [0]:
algo = tpe.suggest

spark_trials = SparkTrials()

with mlflow.start_run():
    best_result = fmin(
        fn = objective, 
        space = search_space,
        algo = algo,
        max_evals = 32,
        trials = spark_trials)

Because the requested parallelism was None or a non-positive value, parallelism will be set to (4), which is Spark's default parallelism (4), or 1, whichever is greater. We recommend setting parallelism explicitly to a positive value because the total of Spark task slots is subject to cluster sizing.
Hyperopt with SparkTrials will automatically track trials in MLflow. To view the MLflow experiment associated with the notebook, click the 'Runs' icon in the notebook context bar on the upper right. There, you can view all runs.
To view logs from trials, please check the Spark executor logs. To view executor logs, expand 'Spark Jobs' above until you see the (i) icon next to the stage from the trial job. Click it and find the list of tasks. Click the 'stderr' link for a task to view trial logs.
100%|██████████| 32/32 [01:47<00:00,  3.37s/trial, best loss: -0.598019801980198]
Total Trials: 32: 32 succeeded, 0 failed, 0 cancelled.


Using hyperopt.space_eval to display the results of the hyperparameter search.

In [0]:
import hyperopt

print(hyperopt.space_eval(search_space, best_result))

{'criterion': 'gini', 'max_depth': 17, 'n_estimators': 400, 'type': 'rf'}
